In [45]:
import datetime
from pathlib import Path

from IPython.display import Markdown as md
from mini_transcript_search import ModelHandler


def dmd(text):
    display(md(text))


handler = ModelHandler(
    use_local_model=False,
    override_stored=False,
    storage_path=Path("..", "data", "parlparse_xmls"),
)

search_query = "register of members financial interests"

today = datetime.date.today()
one_month_ago = today - datetime.timedelta(days=30)


# the last week starting yesterday
last_month = ModelHandler.DateRange(start_date=one_month_ago, end_date=today)
results = handler.query(
    search_query,
    threshold=0.35,
    date_range=last_month,
    chamber=ModelHandler.Chamber.COMMONS,
    transcript_type=ModelHandler.TranscriptType.DEBATES,
)

# dump csv
df = results.df()


100%|██████████| 23/23 [00:07<00:00,  3.23it/s]


# Vector search: register of interests declarations



In [46]:
dmd(f"Last run date: {today}")

Last run date: 2024-09-26

In [44]:
# atched_text	speaker_name	person_id	chamber	transcript_type	speech_id	twfy_ur
df["date"] = df["speech_id"].apply(lambda x: x.split("/")[2][:10])
df["int_person_id"] = df["person_id"].apply(lambda x: x.split("/")[-1] if x else "")

# sort by reverse date
df = df.sort_values("date", ascending=False)

for date, gdf in df.groupby("date", sort=False):
    gdf = gdf.sort_values("distance", ascending=True)
    dmd(f"# {date}")
    for _, row in gdf.iterrows():
        dmd(f"## {row.speaker_name} ({row.int_person_id})")
        dmd(
            f"[Debate link]({row.twfy_url}) | [Register link](https://www.theyworkforyou.com/mp/{row.int_person_id}/register)"
        )

        dmd(f"match score: {round(1 - row.distance,2)}")

        dmd(">" + row.matched_text)

# 2024-09-12

## Mims Davies (25330)

[Debate link](https://www.theyworkforyou.com/debates/?id=2024-09-12b.1057.2) | [Register link](https://www.theyworkforyou.com/mp/25330/register)

match score: 0.77

>Let me first refer Members to my registered interests before I make some points about my constituency

...

>Banking hubs are in many ways very helpful. However, will the Minister consider reviewing the criteria? I have an issue in my constituency where the distance of the nearest bank is one tenth of a mile too short to get a banking hub. Because of that, they are considering not putting a banking hub in place. Is there potential to review the criteria, to support the most vulnerable people in our communities?

## David Smith (26606)

[Debate link](https://www.theyworkforyou.com/debates/?id=2024-09-02a.130.0) | [Register link](https://www.theyworkforyou.com/mp/26606/register)

match score: 0.65

>I thank my hon. Friend for securing the debate on an issue that is important in both his constituency and mine. I draw attention to the correlated issue of post offices. In Wooler in North Northumberland, where there are no bank branches, the post office, which provides the only banking services for that community, is also at risk of closure. Thankfully, an incredible community response, led by the Glendale Gateway Trust, is fighting to retain it. I will do everything in my power, too. Does my hon. Friend agree that banking hubs more generally, and the Post Office specifically, must be part of the solution to the lack of access to banking services in Northumberland?